In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [145]:
# dataset is taken from Kaggle
df_train = pd.read_csv('./dataset/train.csv')
df_test = pd.read_csv('./dataset/test.csv')
df = pd.concat([df_train,df_test], sort=False)

In [146]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1.0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1.0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1.0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1.0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1.0


In [147]:
total_size = len(df)
print('total size: {} rows'.format(total_size))
print('train size: {}, which is {}%'.format(df_train.shape, round(len(df_train)/ total_size  * 100,2)))
print('test size: {}, which is {}%'.format(df_test.shape, round(len(df_test) / total_size * 100,2)))

total size: 10876 rows
train size: (7613, 5), which is 70.0%
test size: (3263, 4), which is 30.0%


In [148]:
df.keyword.value_counts()

wild%20fires             50
survive                  50
trauma                   50
hellfire                 50
desolation               50
collapse                 50
oil%20spill              50
flames                   50
screamed                 50
destroyed                50
terrorist                50
ablaze                   50
weapons                  50
body%20bagging           50
storm                    50
crashed                  50
emergency                50
flooding                 50
rescued                  50
destroy                  50
sinkhole                 50
blown%20up               50
tornado                  50
buildings%20burning      50
upheaval                 50
flood                    50
weapon                   50
earthquake               50
displaced                50
detonation               50
                         ..
flattened                50
seismic                  50
tsunami                  50
windstorm                50
blizzard            

In [149]:
df.keyword.isna().sum()

87

In [150]:
print('missing values of keyword {}%'.format(round(df.location.isna().sum() / total_size * 100,2)))

missing values of keyword 33.45%


In [151]:
df.target.value_counts()

0.0    4342
1.0    3271
Name: target, dtype: int64

In [152]:
df.target.isnull().sum()

3263

In [153]:
# there are 3263 cases which are null in the target, therefore, we delete those values

In [154]:
df.dropna(subset=['target'], inplace=True)

In [155]:
df.shape

(7613, 5)

In [156]:
df.target.isnull().sum()

0

In [157]:
df.target.value_counts()

0.0    4342
1.0    3271
Name: target, dtype: int64

In [158]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1.0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1.0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1.0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1.0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1.0


In [159]:
df.location.isna().sum()

2533

we are going to drop the columns which have missing values as well as ID. 
ID does not add any value to the model that we are going to create.

In [160]:
df.drop(['id','location', 'keyword'], axis=1, inplace=True)

In [169]:
df.sample(10)

,text,target
5891,Watch This Airport Get Swallowed Up By A Sands...,1.0
1984,Unions say they are supportive of 'London' yet...,0.0
6271,Crazy storm hit and I'm trapped inside a Hobby...,1.0
2371,@BV Bloomberg will publish anything negative t...,0.0
6484,We may be just a week away from school but I d...,0.0
6519,Now realized I honestly can't survive without ...,0.0
7603,Officials say a quarantine is in place at an A...,1.0
4415,#hot Funtenna: hijacking computers to send da...,1.0
3731,@mcnabbychic I fear not for a while,0.0
4150,I don't pray harm on members of ISIS.I pray th...,0.0


In [164]:
df.shape

(7613, 2)

In [170]:
df.iloc[2371,:]

text      @BV Bloomberg will publish anything negative t...
target                                                    0
Name: 2371, dtype: object

In [ ]:
# to be continued!!!